## Coursera - Applied Data Science Capstone - Week 3 assignment

### First part: create a Toronto dataframe from the web page

In [1]:
#import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
#import json # library to handle JSON files
import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
#import matplotlib.cm as cm
#import matplotlib.colors as colors
#from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

#### Input data source from the web page

In [2]:
#input data source
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [3]:
#grab the table in the web page
table = soup.find('tbody')
#print(table.prettify())

#### Transform the data into a pandas dataframe

In [4]:
#grab the column name of the table
header = table.tr.text.split('\n')
header

['', 'Postcode', 'Borough', 'Neighbourhood', '']

In [5]:
"""
Write a function that get rid of all the empty strings in the list, 
this will be useful in the following operations
"""
def no_emtpy_str(ls):
    ret = []
    for ele in ls:
        if ele != '':
            ret.append(ele)
    return ret

In [6]:
#remove qll empty strings
header = no_emtpy_str(header)
header

['Postcode', 'Borough', 'Neighbourhood']

In [7]:
# instantiate the dataframe
toronto_df = pd.DataFrame(columns=header)
toronto_df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [8]:
#get the body of the table
body = table.find_all('tr')
#print(body)

In [9]:
#pop off the column name, which is the first row
body.pop(0)
#print(body)

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>

In [10]:
#Take a look at the structure of a single row
print(body[0].text)
print(body[0].text.split('\n'))


M1A
Not assigned
Not assigned

['', 'M1A', 'Not assigned', 'Not assigned', '']


In [11]:
#insert each row into the datframe
for data in body:
    row = data.text.split('\n')
    if row[2] == 'Not assigned':
        pass
    else:
        if row[3] == 'Not assigned':
            #use row[2] to replace row[3]
            toronto_df = toronto_df.append({'Postcode': row[1],
                              'Borough': row[2],
                              'Neighbourhood': row[2]}, ignore_index=True)
        else:
            toronto_df = toronto_df.append({'Postcode': row[1],
                                          'Borough': row[2],
                                          'Neighbourhood': row[3]}, ignore_index=True)
    
toronto_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [12]:
toronto_df.shape

(211, 3)

### Second Part: Add geographical coordinates into the Toronto dataframe

#### Import the geographical coordinates for Toronto

In [13]:
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
coordinates_df.head(100)
#check if matches both dataframe total postcode number matches
print(coordinates_df.shape)
print(toronto_df['Postcode'].unique().shape)

(103, 3)
(103,)


In [14]:
#create a Latitude and Longitude column for toronto_df
latitude = pd.Series([])
longitude = pd.Series([])
for i in range(len(toronto_df)):
    latitude[i] = float(coordinates_df.loc[coordinates_df['Postal Code'] == toronto_df['Postcode'][i]].Latitude)
    longitude[i] = float(coordinates_df.loc[coordinates_df['Postal Code'] == toronto_df['Postcode'][i]].Longitude)

In [15]:
#Add a new column into the toronto dataframe
toronto_df['Latitude'] = latitude
toronto_df['Longitude'] = longitude
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


### Visualizing the data and clustering

#### Visualizing

In [16]:
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

In [18]:
# create map of Toronto latitude and longitude values
map_toronto = folium.Map(location=[43.753259, -79.329656], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [19]:
# @hidden_cell
print('FUCK')

FUCK
